## Stuff document summarization

In [2]:
pip install ipykernel

Note: you may need to restart the kernel to use updated packages.


In [3]:
#define the LLM model

##Tools , LLM >>Agent Executer
from langchain_groq import ChatGroq
from dotenv import  load_dotenv
import os
load_dotenv()

groq_key=os.getenv("GROQ_API_KEY")
llm=ChatGroq(groq_api_key=groq_key,model="llama-3.3-70b-versatile",temperature=0.1)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020B4C8BD160>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020B4C8BF4D0>, model_name='llama-3.3-70b-versatile', temperature=0.1, model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_community.document_loaders import PyPDFLoader
from langchain.prompts import PromptTemplate

loader=PyPDFLoader("apjspeech.pdf")
doc=loader.load_and_split()
doc

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth

In [5]:
template="""Write the concise and short summery of the following speech
:{text}
"""
prompt=PromptTemplate(
    input_variable=["text"],
    template=template)



In [6]:
from langchain.chains.summarize import  load_summarize_chain

In [7]:
chain=load_summarize_chain(
    llm,
    chain_type="stuff",
    prompt=prompt,
    verbose=True)
output_summery=chain.run(doc)
output_summery

C:\Users\Nikhil Warge\AppData\Local\Temp\ipykernel_12840\473422233.py:6: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_summery=chain.run(doc)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write the concise and short summery of the following speech
:A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactio

"Here is a concise and short summary of the speech:\n\nIn his departing speech as President of India, A.P.J. Abdul Kalam reflects on his five-year tenure and highlights 10 key messages for the nation's development. He emphasizes the importance of empowering villages, mobilizing rural core competence, and promoting connectivity for societal transformation. He also stresses the need to defeat problems, overcome disasters, and defend the nation with courage and pride. Kalam expresses his confidence in the youth of India, who he believes can lead the country to become a developed nation by 2020. He concludes by resolving to work towards a developed India with a distinctive profile, where there is equitable distribution of resources, quality education, and good governance, and calls upon citizens to join him in this mission."

## Map Reduce Summarization


In [8]:
#MapReduce summerization

from langchain_text_splitters import RecursiveCharacterTextSplitter


In [9]:
final_document=RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100
).split_documents(doc)
final_document

[Document(metadata={'producer': 'GPL Ghostscript 8.15', 'creator': 'PScript5.dll Version 5.2', 'creationdate': 'D:20070730160943', 'moddate': 'D:20070730160943', 'title': 'Microsoft Word - Document1', 'author': 'Shri', 'source': 'apjspeech.pdf', 'total_pages': 7, 'page': 0, 'page_label': '1'}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth

In [10]:
len(final_document)

22

In [11]:
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='Write the concise and short summery of the following speech\n:{text}\n')

In [12]:
chunk_prompt="""
Please summarise the below speech: {text}
symmary:
"""
map_prompt_template=PromptTemplate(input_variables=["text"],template=chunk_prompt)


In [13]:
#2nd prompt for the final output

prompt_template="""
Provide the summery of the entire speech with these important points.
Add the motivational title,start the precise summary with an instruction and provide the summery in number points of the speech.
Speech:{text}
"""
final_prompt_template=PromptTemplate(input_variables=["text"],template=prompt_template)


In [14]:
summery_chain=load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=final_prompt_template,
    verbose=True
)
output=summery_chain.run(final_document)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarise the below speech: A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have many uniqu

c:\Users\Nikhil Warge\python_code\Creating_tool_using_langchain\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
c:\Users\Nikhil Warge\python_code\Creating_tool_using_langchain\venv\Lib\site-packages\huggingface_hub\file_download.py:144: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Nikhil Warge\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details



> Entering new LLMChain chain...
Prompt after formatting:

Provide the summery of the entire speech with these important points.
Add the motivational title,start the precise summary with an instruction and provide the summery in number points of the speech.
Speech:The speech is a farewell address by A.P.J. Abdul Kalam, the former President of India, after completing his 5-year term at Rashtrapati Bhavan. He expresses his gratitude to the people of India and those living abroad for their support and association during his tenure. He reflects on his interactions with people from various walks of life, including politics, science, arts, and youth, and highlights the importance of certain messages, starting with "Accelerate development: Aspiration of the youth". The speech is a thanksgiving occasion where he shares his unique experiences and lessons learned during his time in office.

The speech highlights 10 important messages for the development and growth of India. These messages incl

'**Motivational Title: "Empowering a Developed India: A Vision for the Future"**\n\nTo understand the key takeaways from the speech, follow this instruction: **Read the summary points carefully and reflect on how they can be applied to your own life and community.**\n\nHere is a precise summary of the speech in numbered points:\n\n1. **Accelerate development**: The speech emphasizes the importance of accelerating development to meet the aspirations of India\'s youth, with a focus on empowering villages and utilizing rural core competence for competitiveness.\n2. **Empowering villages**: The speaker highlights the need to connect rural villages to urban societies through quality roads and other amenities, citing the example of the Periyar Maniammai College of Technology for Women\'s initiative to provide urban amenities in rural areas (PURA).\n3. **Agriculture and growth**: The speech discusses the importance of focusing on agriculture for growth, using the "Seed to Food" principle, and

In [15]:
summery_chain

MapReduceDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nPlease summarise the below speech: {text}\nsymmary:\n'), llm=ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000020B4C8BD160>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000020B4C8BF4D0>, model_name='llama-3.3-70b-versatile', temperature=0.1, model_kwargs={}, groq_api_key=SecretStr('**********')), output_parser=StrOutputParser(), llm_kwargs={}), reduce_documents_chain=ReduceDocumentsChain(verbose=True, combine_documents_chain=StuffDocumentsChain(verbose=True, llm_chain=LLMChain(verbose=True, prompt=PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide the summery of the entire speech with these important points.\nAdd the motivational title,start the precise summary with an instruction and provide the summery in number

## Refine Summarization

In [16]:
chain=load_summarize_chain(
    llm=llm,
    chain_type="refine",
    verbose=True,
)
output_cahin=chain.run(final_document)

output_cahin



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have many

"The original summary is comprehensive and effectively conveys the key points of APJ Abdul Kalam's farewell speech. The additional context provided does not significantly alter the main themes or messages presented in the speech. It primarily consists of a concluding statement, a personal mission, and a final farewell message. \n\nTherefore, the original summary remains refined and accurate, and there is no need to revise it based on the provided context. The original summary is:\n\nAPJ Abdul Kalam delivers a farewell speech after 5 years at Rashtrapati Bhavan, expressing gratitude and sharing experiences from his tenure, highlighting 10 key messages including accelerating development, empowering villages, and a youth movement for a Developed India 2020. These messages were inspired by interactions with the youth, such as a question from a young girl named Anukriti who asked why India cannot become a developed nation before 2020. He also drew inspiration from his visit to Nagaland, whe